In [12]:
# encoding: utf-8

import numpy as np


class Helmholtz():

    def __init__(self, eq, w, Tc, Pc, Tr, R):
        self.eq = eq
        self.w = w
        self.Tc = Tc
        self.Pc = Pc
        self.Tr = Tr
        self.R = R
        if self.eq == 1:
            # Soave-Redlich-Kwong (SRK)
            self.s1, self.s2 = 1, 2
            self.m = 0.480 + 1.574 * self.w - 0.175 * self.w ** 2
            self.ac = 0.077796070 * self.R ** 2, self.Tc ** 2 / self.Pc
            self.bc = 0.086640 * self.R * self.Tc / self.Pc
        elif self.eq == 2:
            # Peng-Robinson (PR)
            self.s1, self.s2 = 1 + 2 ** 0.5, 1 - (2 ** 0.5)
            self.m = 0.37464 + 1.54226 * self.w - 0.26992 * self.w ** 2
            self.ac = 0.45723553 * self.R ** 2 * self.Tc ** 2 / self.Pc
            self.bc = 0.077796070 * self.R * self.Tc / self.Pc        
        else:
            print ("Che boludo... Modelo no valido, intentaló de nuevo !!! ")
        print (("m = ", self.m))
        print (("ac = ", self.ac))
        print (("bc = ", self.bc))

    def parametros(self, ni, nT, nC, V, T):
        self.ni = ni
        self.nT = nT
        self.nC = nC
        self.V = V
        self.T = T
        self.alfa = (1 + self.m * (1 - (self.T / self.Tc) ** 0.5)) ** 2
        self.dalfadT = - (self.m / self.T) * (self.T / self.Tc) ** 0.5 * (self.m * (- (self.T / self.Tc) ** 0.5 + 1) + 1)
        ter_1 = 0.5 * self.m ** 2 * (self.T / self.Tc) ** 1.0 / self.T ** 2
        ter_2 = 0.5 * self.m * (self.T / self.Tc) ** 0.5 * (self.m * (- (self.T / self.Tc) ** 0.5 + 1) + 1) / self.T ** 2
        self.d2alfaT2 = ter_1 + ter_2
        self.a_ii = self.ac * self.alfa
        self.b_ii = self.bc
        self.da_iidT = self.ac * self.dalfadT
        d2adT2_puros = self.ac * self.d2alfaT2

        if self.nC > 1:
            self.aij = np.ones((len(self.ni), len(self.ni)))
            self.bij = np.ones((len(self.ni), len(self.ni)))
            self.daijdT = np.ones((len(self.ni), len(self.ni)))

            for j in range(self.nC):
                for i in range(self.nC):
                    self.aij[i, j] = (self.a_ii[i] * self.a_ii[j]) ** 0.5
                    self.bij[i, j] = (self.b_ii[i] + self.b_ii[j]) / 2
                    self.bij[i, j] = self.bij[i, j]
                    self.daijdT[i, j] = (self.da_iidT[i] * self.da_iidT[j]) ** 0.5

            for i in range(self.nC):
                for  j in range(self.nC):
                    if i == j:
                        self.aij[i, j] = self.a_ii[i] * (1 - kij[i, j])
                        self.daijdT[i, j] = self.da_iidT[i] * (1 - kij[i, j])
                    elif i != j:
                        self.aij[i, j] = self.aij[i, j] * (1 - kij[i, j])
                        self.daijdT[i, j] = self.daijdT[i, j] * (1 - kij[i, j])

        if self.nC == 1:
            return self.a_ii, self.b_ii, self.da_iidT
        else:
            return self.aij, self.bij, self.daijdT

    def parametro_D(self):
        if self.nC == 1:
            self.D = self.ni ** 2 * self.a_ii
            self.Di = 2 * self.ni * self.a_ii
        elif self.nC > 1:
            di = np.ones((len(self.ni), len(self.ni)))
            self.Di = np.ones((len(self.ni)))
            self.D = np.ones((len(self.ni)))
            for i in range(self.nC):
                for j in range(self.nC):
                    di[i, j] = self.ni[j] * self.aij[i, j]
                    self.Di[i] = 2 * np.sum(di[i, :])
            self.D = 0.5 * np.sum(ni * self.Di)

        return self.D

    def parametro_B(self):
        if self.nC == 1:
            self.B = self.ni * self.b_ii
        elif self.nC > 1:
            self.aux = np.zeros((len(self.ni)))
            for i in range(self.nC):
                for j in range(self.nC):
                    self.aux[i] = self.aux[i] + self.ni[j] * self.bij[i, j]

            self.B = np.sum(self.ni * self.b_ii)

        return self.B

    def presion(self):
        '''
        Con el método presion(), se calcula la Presión P(T, V, N) del sistema
        para una temperatura T, cantidad de moles N y un volumen V
        R = Constante universal de los gases
        nT = Número total de moles en el sistema
        Pcal = Presión calculada con la ecuación de estado
        Arv = Primera derivada parcial de la energía de Helmholz con respecto al
        volumen V, a T y N constantes
        '''
        self.gv = self.R * self.B / (self.V * (self.V - self.B))
        self.fv = - 1 / ((self.V + self.s1 * self.B) * (self.V + self.s2 * self.B))
        self.ArV = -self.nT * self.gv * self.T - self.D * self.fv
        self.Pcal = self.nT * self.R * self.T / self.V - self.ArV
        return self.Pcal

    def dP_dV(self):
        self.dPdV = -self.ArV2 - self.R * self.T * self.nT / self.V ** 2
        return self.dPdV

    def Z_factor(self, P):
        self.P = P
        self.Z = (self.P * self.V) / (self.nT * self.R * self.T)
        return self.Z

    def P_ideal(self, P):
        self.P = P
        self.Pxi = (self.ni * self.P) / self.nT
        return self.Pxi

    def dF_dV(self):
        '''
        Primera derivada de F con respecto al volumen Ecu. (68)
        '''
        self.gv = self.R * self.B / (self.V * (self.V - self.B))
        self.fv = - 1 / ((self.V + self.s1 * self.B) * (self.V + self.s2 * self.B))
        self.ArV = -self.nT * self.gv * self.T - self.D * self.fv
        return self.ArV

    def dF_dVV(self):
        '''
        Segunda derivada de F con respecto al volumen Ecu. (74)
        '''
        self.gv2 = self.R * (1 / self.V ** 2 - 1 / (self.V - self.B) ** 2)
        self.fv2 = (- 1 / (self.V + self.s1 * self.B) ** 2 + 1 / (self.V + self.s2 * self.B) ** 2) / self.B / (self.s1 - self.s2)
        self.ArV2 = - self.nT * self.gv2 * self.T - self.D * self.fv2
        return self.ArV2

    def volumen_1(self, P):
        '''
        Calculo del volumen V(T,P,n) del fluido a una temperatura T, presión P
        y número de moles totales nT especificados.
        Se utiliza el método de Newton con derivada de la función analitica.
        Pendiente cambiar por una función de Scipy.
        '''
        self.P = P
        self.V = 1.05 * self.B
        lnP = np.log(self.P)
        print "P_esp = ", self.P
        print "V_ini = ", self.V
        Pite = self.presion()
        lnPcal = np.log(Pite)
        #h = self.P - Pite
        h = lnP - lnPcal
        errorEq = abs(h)
        print "ErrorP = ", errorEq
        i = 0
        s = 1.0

        while errorEq > ep:
            self.parametro_D()
            self.parametro_B()
            self.dF_dV()
            self.dF_dVV()
            dPite = self.dP_dV()
            Pite = self.presion()
            lnPcal = np.log(Pite)
            h = lnP - lnPcal
            dh = -dPite
            self.V = self.V - s * h / dh
            errorEq = abs(h)
            
            i += 1
            if i >= 2000:
                pass
                #break
        print "FV = ", dPite
        print "i = ", i

        return self.V

   

    def primeras_derivadas1(self):

        if nC == 1:
            AUX = self.R * self.T / (self.V - self.B)
            self.fB = -(self.f + self.V * self.fv) / self.B
            self.FFB = self.nT * AUX - self.D * self.fB
            self.Di = 2 * self.nT * self.ac * self.alfa
            self.Bi = self.bc
            self.Arn = -self.g * self.T + self.FFB * self.Bi - self.f * self.Di
        elif nC >= 1:
            # Derivando la ecuación (64) se obtiene la ecuación eq (106)
            self.Bi = np.ones((len(self.ni)))
            for i in range(nC):
                self.Bi[i] = (2 * self.aux[i] - self.B) / self.nT

            AUX = self.R * self.T / (self.V - self.B)
            self.fB = -(self.f + self.V * self.fv) / self.B
            self.FFB = self.nT * AUX - self.D * self.fB
            self.Arn = -self.g * self.T + self.FFB * self.Bi - self.f * self.Di

        print "Bi = ", self.Bi
        print "Di = ", self.Di
        print "fB = ", self.fB
        print "FFB = ", self.FFB
        print "Arn cal = ", self.Arn

        return self.Arn, self.Arn, self.Arn

    def fluido(self, P):
        self.P = P
        ab = self.parametros(self.ni, self.nT, self.nC, self.V, self.T)
        print "................................................................"
        print (("aij = ", ab[0]))
        print (("bij = ", ab[1]))
        print "................................................................"
        D = self.parametro_D()
        B = self.parametro_B()
        print (("D = ", D))
        print (("B = ", B))
        print "................................................................"
        Vol_1 = self.volumen_1(self.P)
        print (("Volumen_1 = ", Vol_1))
        print (("Densidad =", 1 / Vol_1))
        print "................................................................"
        
        return Vol_1
    
#--------------------------- Númuro de componentes -----------------------------
nC = 3
#---------------------------- Temperatura en K ---------------------------------
# K
T = 368.0
#T = 100.0
#-------------------------- Presión --------------------------------------------
# Bar
P = 800.0
#P = 4.12
#--------------------------- Volumen ------------------------------------------
#--------------- Constante R [=] # bar.l/(mol.K) : 0.08314472-------------------
# bar.l/(mol.K) : 0.08314472
R = 0.08314472
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
# selección de la Ecuación de Estado
# eq = 1, para Ecuación de Estado (SRK)
# eq = 2, para Ecuación de Estado (PR)
eq = 2
#------------------ Criterio de convergencia en línea 215 ---------------------
#------------------ del método def volumen_1(self, P): ------------------------
ep = 1e-6
#------------------------------------------------------------------------------
#--------------------------- Fugacidad Fluido Puro ----------------------------
#------------------------------------------------------------------------------
print "..................................................................."

# metano - propano - C24
Tcm = np.array([190.56, 369.83, 804.0])
Pcm = np.array([45.99, 41.924, 9.8]) #9.672
wm = np.array([0.0115, 0.1523, 1.071])

# metano - C24
#Tcm = np.array([190.56, 804.0])
#Pcm = np.array([45.99, 9.8])
#wm = np.array([0.0115, 1.071])

# C3 - iC4 - nC4
#Tcm = np.array([369.8, 408.1, 425.2])
#Pcm = np.array([42.49, 36.48, 37.97])
#wm = np.array([0.152, 0.177, 0.193])

# N2 - C1
#Tcm = np.array([126.1, 190.56])
#Pcm = np.array([33.94, 45.99])
#wm = np.array([0.040, 0.0115])


print "..............................................................."
# C3 - iC4 - nC4
#ni = np.array([0.23, 0.67, 0.10])

# (C1, C3, C24)
ni = np.array([0.8224, 0.0859, 0.0917])

# (C1, C24)
#ni = np.array([0.3, 0.7])

# (N2, C1)
#ni = np.array([0.958, 1-0.958])

# (C1, C3, C24)
kij = np.array([[0.000000, 0.016715, 0.083860],
                [0.016715, 0.000000, 0.032225],
                [0.083860, 0.032225, 0.000000]])

#kij = np.array([[0.000000, 0.083860],
#                [0.083860, 0.000000]])

#kij = np.array([[0.000000, 0.000000, 0.000000],
#                [0.000000, 0.000000, 0.000000],
#                [0.000000, 0.000000, 0.000000]])

#kij = np.array([[0.000000, 0.0242398434],
#                [0.0242398434, 0.000000]])

lij = 0.0

# metano - propano - C24
Tc = Tcm
Pc = Pcm
w = wm
print "..............................................................."

#---------------------------------------------------------------------------
# Tempertura reducidad
Tr = T / Tc
# C24 puro
V = 0.141604834257319
nT = np.sum(ni)

print "..................................................................."
calculo = Helmholtz(eq, w, Tc, Pc, Tr, R)
ab = calculo.parametros(ni, nT, nC, V, T)
#print ab

flu_1 = calculo.fluido(P)





...................................................................
...............................................................
...............................................................
...................................................................
('m = ', array([ 0.39234029,  0.60326533,  1.71679115]))
('ac = ', array([   2.49579781,   10.31218946,  208.49485499]))
('bc = ', array([ 0.0268016 ,  0.05706   ,  0.53066728]))
................................................................
('aij = ', array([[   1.79102083,    4.23207833,   27.53441889],
       [   4.23207833,   10.34303347,   69.89757599],
       [  27.53441889,   69.89757599,  504.34474715]]))
('bij = ', array([[ 0.0268016 ,  0.0419308 ,  0.27873444],
       [ 0.0419308 ,  0.05706   ,  0.29386364],
       [ 0.27873444,  0.29386364,  0.53066728]]))
................................................................
('D = ', 11.380720543934359)
('B = ', 0.075605276167156371)
...................................